# **Import and Install**

In [ ]:
!pip install torch torchvision transformers

     |████████████████████████████████| 4.0 MB 15.2 MB/s 
     |████████████████████████████████| 880 kB 55.4 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
     |████████████████████████████████| 596 kB 53.3 MB/s 
     |████████████████████████████████| 77 kB 4.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e1b8019793c2a48c78603da89701c41f2b3792c8cc39a7883a4717ac6a93fdc5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import nltk
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from transformers import AdamW
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch.optim as optim
import time

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from LoadData import LoadDataAndProcessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "/content/output"

# **Build The Model**

In [ ]:
class RumourDataset(Dataset):

    def __init__(self, data, maxlen):

        self.data = data

        if maxlen > 512:
            self.maxlen = 512
        else:
            self.maxlen = maxlen

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text_source, text_reply = self.data[index]['text_source'], self.data[index]['text_reply']

        if len(self.data[index]) == 3:
          label = self.data[index]['label']
        else:
          label = None

        # Preprocessing the text to be suitable for BERT
        # Insering the CLS and SEP token in the beginning and end of the source and replies
        tokens = ["[CLS]"]
        tokens_source = self.tokenizer.tokenize(text_source) # Tokenize the sentence
        tokens += tokens_source + ["[SEP]"]
        len_source = len(tokens)

        len_reply = 0
        if len_source < self.maxlen:
            tokens_reply = self.tokenizer.tokenize(text_reply)[:self.maxlen - len_source - 1]
            tokens += tokens_reply + ["[SEP]"]
            len_reply = len(tokens) - len_source

        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        if len(self.data[index]) == 3:
          return tokens_ids_tensor, attn_mask, label
        else:
          return tokens_ids_tensor, attn_mask

In [ ]:
train_file = "/content/drive/MyDrive/NLP/data/train_data_prepared.json"
dev_file = "/content/drive/MyDrive/NLP/data/dev_data_prepared.json"

load_twitter_train_data = LoadDataAndProcessing(train_file)
load_twitter_dev_data = LoadDataAndProcessing(dev_file)

train_input = load_twitter_train_data.prepareDataset()
dev_input = load_twitter_dev_data.prepareDataset()

In [ ]:
#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = RumourDataset(data = train_input, maxlen = 512)
dev_set = RumourDataset(data = dev_input, maxlen = 512)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 8, num_workers = 2, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size = 8, num_workers = 2, shuffle=True)

print("Done preprocessing training and development data.")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Done preprocessing training and development data.


In [ ]:
class RumorClassifier(nn.Module):

    def __init__(self):
        super(RumorClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [ ]:
gpu = 0 #gpu ID

print("Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...")
model = RumorClassifier()
model.cuda(gpu)  #Enable gpu support for the model
print("Done creating the rumour classifier.")

Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the rumour classifier.


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 2e-5, eps=1e-8)

# **Trian and Evaluate the Model**

In [ ]:
def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc= 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 200 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            print("-----------------------------------------------------------------")
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))


In [ ]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()

    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
num_epoch = 6

#fine-tune the model
train(model, criterion, optimizer, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.699819028377533; Accuracy: 0.625; Time taken (s): 1.0799386501312256
Epoch 0 complete! Development Accuracy: 0.8584558963775635; Development; Development Loss: 0.30342722443096776
Best development accuracy improved from 0 to 0.8584558963775635, saving model...
-----------------------------------------------------------------
Iteration 0 of epoch 1 complete. Loss: 0.190723717212677; Accuracy: 1.0; Time taken (s): 153.94081664085388
Epoch 1 complete! Development Accuracy: 0.9209558963775635; Development; Development Loss: 0.2150109550326734
Best development accuracy improved from 0.8584558963775635 to 0.9209558963775635, saving model...
-----------------------------------------------------------------
Iteration 0 of epoch 2 complete. Loss: 0.047463640570640564; Accuracy: 1.0; Time taken (s): 163.3366882801056
Epoch 2 complete! Development Accuracy: 0.9319853186607361; Development; Development Loss: 0.15958736216485062
Best development accuracy imp

# **Devset Prediction**

In [ ]:
def predict(net, test_loader, model_file=None):
    # load weight
    if model_file != None:
      net.load_state_dict(torch.load(model_file))
    
    predictions = []
    
    # Predict process
    with torch.no_grad():
        for seq, attn_masks in test_loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()
            predictions.extend(soft_probs.cpu().numpy().squeeze())
    
    return predictions

In [ ]:
dev_to_predict_input_file = "/content/drive/MyDrive/NLP/data/dev_data_to_predict.json"

load_twitter_dev_to_predict_data = LoadDataAndProcessing(dev_to_predict_input_file)

dev_to_predict_input = load_twitter_dev_to_predict_data.prepareDataset()

dev_to_predict_set = RumourDataset(data = dev_to_predict_input, maxlen = 512)
dev_to_predict_loader = DataLoader(dev_to_predict_set, batch_size = 8, num_workers = 2)

In [ ]:
model_file = "sstcls_5.dat"
dev_prediction = predict(model, dev_to_predict_loader, model_file)
len(dev_prediction)

538

In [ ]:
dev_pred = pd.DataFrame(data=dev_prediction, columns=['Predicted']).reset_index()
dev_pred = dev_pred.rename(columns={'index': 'Id'})
print(dev_pred)

dev_pred.to_csv('dev_prediction.csv', index = False)

      Id  Predicted
0      0          0
1      1          0
2      2          0
3      3          0
4      4          0
..   ...        ...
533  533          1
534  534          0
535  535          1
536  536          0
537  537          0

[538 rows x 2 columns]


In [ ]:
dev_true_label_file = '/content/drive/MyDrive/NLP/data/valid.dev.label.txt'
dev_true_label = pd.read_table(dev_true_label_file,sep='\n', 
                               header=None)[0].apply(lambda x: 1 if x=="rumour" else 0)

In [ ]:
y_true = list(dev_true_label)
y_pred = dev_prediction

In [ ]:
p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, pos_label=1, average="binary")
a = accuracy_score(y_true, y_pred)

In [ ]:
print("Performance on the rumour class:")
print("Accuracy  =", a)
print("Precision =", p)
print("Recall    =", r)
print("F1        =", f1)

Performance on the rumour class:
Accuracy  = 0.9368029739776952
Precision = 0.8188976377952756
Recall    = 0.9043478260869565
F1        = 0.859504132231405


In [ ]:
with open('output/dev_performance.txt', 'w') as f:
    f.write("Accuracy  = {}\n".format(a) +
            "Precision = {}\n".format(p) +
            "Recall    = {}\n".format(r) + 
            "F1        = {}\n".format(f1))

# **Testset Prediction**

In [ ]:
test_input_file = "/content/drive/MyDrive/NLP/data/test_data_prepared.json"

load_twitter_test_data = LoadDataAndProcessing(test_input_file)

test_input = load_twitter_test_data.prepareDataset()

test_set = RumourDataset(data = test_input, maxlen = 512)
test_loader = DataLoader(test_set, batch_size = 8, num_workers = 2)

In [ ]:
model_file = "sstcls_5.dat"
test_prediction = predict(model, test_loader, model_file)
len(test_prediction)

558

In [ ]:
test_pred = pd.DataFrame(data=test_prediction, columns=['Predicted']).reset_index()
test_pred = test_pred.rename(columns={'index': 'Id'})
print(test_pred)

test_pred.to_csv('test_prediction_bert.csv', index = False)

      Id  Predicted
0      0          0
1      1          1
2      2          0
3      3          0
4      4          1
..   ...        ...
553  553          0
554  554          0
555  555          1
556  556          0
557  557          0

[558 rows x 2 columns]
